In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Get the HTML content from the page
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Example: Extract data from a table on the page
# Find the table(s) you're interested in; adjust as needed
table = soup.find('table', {'class': 'wikitable'})

# Convert the table to a DataFrame
df = pd.read_html(str(table))[0]

print(df.head())


   Flight No.                                Date and time (UTC)  \
0           1                                 4 June 2010, 18:45   
1           1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2           2                         8 December 2010, 15:43[13]   
3           2  Maiden flight of Dragon capsule, consisting of...   
4           3                             22 May 2012, 07:44[17]   

                                Version, Booster [b]  \
0                              F9 v1.0[7] B0003.1[8]   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                              F9 v1.0[7] B0004.1[8]   
3  Maiden flight of Dragon capsule, consisting of...   
4                              F9 v1.0[7] B0005.1[8]   

                                         Launch site  \
0                                      CCAFS, SLC-40   
1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2                                      CCAFS, SLC-40   
3  Maiden flight of Dragon cap

C:\Users\Joe_Demi\AppData\Local\Temp\ipykernel_15668\3925621588.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


# 1. After you performed a GET request on the Space X API and convert the response  to a dataframe using pd.json_normalize. What year is located in the first row in the column static_fire_date_utc?

In [9]:
import pandas as pd
import requests

# URL of the SpaceX API endpoint
api_url = "https://api.spacexdata.com/v4/launches"  # Example API endpoint

# Perform the GET request
response = requests.get(api_url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

# Extract the static_fire_date_utc value from the first row
static_fire_date_utc = df.loc[0, 'static_fire_date_utc']

# Convert the date to a datetime object and extract the year
year = pd.to_datetime(static_fire_date_utc).year

print(year)


2006


# 2. Using the API, how many Falcon 9 launches are there after we remove Falcon 1 launches?

In [13]:
import pandas as pd
import requests

# URL of the SpaceX API endpoint
api_url = "https://api.spacexdata.com/v4/launches"

# Perform the GET request
response = requests.get(api_url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

# Display the first few rows and column names to inspect
print(df.head())
print(df.columns)

# Assuming 'rocket' is a nested field, extract relevant columns if needed
# You might need to access nested fields if 'rocket' is a dict
if 'rocket' in df.columns:
    df['rocket_name'] = df['rocket'].apply(lambda x: x['name'] if isinstance(x, dict) else None)

# Check the DataFrame again to ensure 'rocket_name' is correctly added
print(df.head())

# Filter out Falcon 1 launches
df_falcon_1 = df[df['rocket_name'] == 'Falcon 1']

# Filter out Falcon 9 launches
df_falcon_9 = df[df['rocket_name'] == 'Falcon 9']

# Remove Falcon 1 launches from the DataFrame
df_filtered = df[~df['rocket_name'].isin(df_falcon_1['rocket_name'])]

# Count the remaining Falcon 9 launches
count_falcon_9 = df_filtered['rocket_name'].value_counts().get('Falcon 9', 0)

print(f"Number of Falcon 9 launches after removing Falcon 1 launches: {count_falcon_9}")


       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                            failures  \
0  [{'time': 33, 'altitude': None, 'reason': 'mer...   
1  [{'time': 301, 'altitude': 289, 'reason': 'har...   
2  [{'time': 140, 'altitude': 35, 'reason': 'resi...   
3                                                 []   
4                                                 []   

             

In [17]:
import requests

# Step 1: Get the page ID
response = requests.get('https://en.wikipedia.org/w/api.php', params={
    'action': 'query',
    'titles': 'List_of_Falcon_9_and_Falcon_Heavy_launches',
    'format': 'json'
})

data = response.json()
page_id = next(iter(data['query']['pages']))  # Get the first key (page ID)

# Step 2: Fetch the content of the page
response = requests.get('https://en.wikipedia.org/w/api.php', params={
    'action': 'query',
    'prop': 'revisions',
    'titles': 'List_of_Falcon_9_and_Falcon_Heavy_launches',
    'rvprop': 'content',
    'format': 'json'
})

data = response.json()
content = data['query']['pages'][page_id]['revisions'][0]['*']

# Example parsing logic
# This will depend on how the data is formatted on the page
# Adjust the parsing according to the actual content

# For demonstration, let's assume you can count 'Falcon 9' and 'Falcon 1' occurrences
falcon_9_launches = content.count('Falcon 9')
falcon_1_launches = content.count('Falcon 1')

# Calculate Falcon 9 launches after removing Falcon 1 launches
falcon_9_after_removal = falcon_9_launches - falcon_1_launches
print(falcon_9_after_removal)


698


In [23]:
import pandas as pd
import requests

# URL of the SpaceX API endpoints
launches_api_url = "https://api.spacexdata.com/v4/launches"
rockets_api_url = "https://api.spacexdata.com/v4/rockets"

# Perform the GET requests
launches_response = requests.get(launches_api_url)
rockets_response = requests.get(rockets_api_url)

# Convert the JSON response to DataFrames
launches_data = launches_response.json()
rockets_data = rockets_response.json()

launches_df = pd.json_normalize(launches_data)
rockets_df = pd.json_normalize(rockets_data)

# Get Falcon 9 and Falcon 1 rocket IDs
falcon_9_id = rockets_df[rockets_df['name'].str.contains('Falcon 9', case=False, na=False)]['id'].values
falcon_1_id = rockets_df[rockets_df['name'].str.contains('Falcon 1', case=False, na=False)]['id'].values

# Check the IDs we obtained
print(f"Falcon 9 IDs: {falcon_9_id}")
print(f"Falcon 1 IDs: {falcon_1_id}")

# Filter the DataFrame for Falcon 9 launches
falcon_9_launches = launches_df[launches_df['rocket'].isin(falcon_9_id)]

# The filtered Falcon 9 launches should not have Falcon 1, but you might want to ensure
# that Falcon 1 launches are not in the Falcon 9 launches DataFrame
# Here, we just count Falcon 9 launches
num_falcon_9_launches = len(falcon_9_launches)

print(f"Number of Falcon 9 launches: {num_falcon_9_launches}")


Columns in DataFrame:
Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number of Falcon 9 launches after removing Falcon 1 launches: 0


# 3. At the end of the API data collection process, how many missing values are there for the column landingPad?

In [15]:
import pandas as pd
import requests

# URL of the SpaceX API endpoint
api_url = "https://api.spacexdata.com/v4/launches"

# Perform the GET request
response = requests.get(api_url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

# Display the column names to find the correct column
print("Columns in DataFrame:")
print(df.columns)

# Check for missing values in the column related to landing pads
# Assuming the column might be something like 'landing_pad' or 'landing_pads'
# Replace 'landing_pad' with the correct column name if needed
if 'landing_pad' in df.columns:
    missing_values_count = df['landing_pad'].isna().sum()
elif 'landing_pads' in df.columns:
    missing_values_count = df['landing_pads'].isna().sum()
else:
    missing_values_count = "Column not found"

print(f"Number of missing values in the landing pad column: {missing_values_count}")


Columns in DataFrame:
Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number of missing values in the landing pad column: Column not found


In [21]:
import pandas as pd
import requests

# URL of the SpaceX API endpoint
api_url = "https://api.spacexdata.com/v4/launches"

# Perform the GET request
response = requests.get(api_url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

# Display the column names to find the correct column
print("Columns in DataFrame:")
print(df.columns)

# Check for missing values in the column related to landing pads
# Assuming the column might be 'landing_pad' or 'landing_pads'
# Replace 'landing_pad' with the correct column name if needed
if 'landing_pad' in df.columns:
    missing_values_count = df['landing_pad'].isna().sum()
elif 'landing_pads' in df.columns:
    missing_values_count = df['landing_pads'].isna().sum()
else:
    missing_values_count = "Column not found"

print(f"Number of missing values in the landing pad column: {missing_values_count}")


Columns in DataFrame:
Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number of missing values in the landing pad column: Column not found


In [22]:
import pandas as pd
import requests

# URL of the SpaceX API endpoint
api_url = "https://api.spacexdata.com/v4/launches"

# Perform the GET request
response = requests.get(api_url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

# Display the column names to find the correct column
print("Columns in DataFrame:")
print(df.columns)

# Check for missing values in the 'launchpad' column
if 'launchpad' in df.columns:
    missing_values_count = df['launchpad'].isna().sum()
else:
    missing_values_count = "Column not found"

print(f"Number of missing values in the launchpad column: {missing_values_count}")


Columns in DataFrame:
Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number of missing values in the launchpad column: 0
